# Numerical Solutions to the Advection Eqaution in 1-D

## Mathematical Problem

The homogenous advection equation in one dimmension is  
\begin{align}
  \frac{\partial u}{\partial t} = a \frac{\partial u}{\partial x}
\end{align}
where $u(x,t)$ is a scalar field (_e.g._ density, enthalpy) and $a$ is the advection velocity. This hyperbolic equation is simple very useful for insight on how to numerically solve hyperbolic PDEs. In order to solve this equation (numerically or analytically) we need inital conidtions at $t_0$  
\begin{align}
  u(x,t_0) = \eta(x)
\end{align}
and boundary conditions  
\begin{align}
\begin{aligned}
  u(0, t) = g_0(t) \;\;\; \text{for} \;\;\; t>0\\
  u(L, t) = g_0(t) \;\;\; \text{for} \;\;\; t>0
\end{aligned}
\end{align}  
on the domain $0<x<L$.

## Numerical Approaches
We will aplly finite differences on a discrete grid with grid points $(x_i, t_j)$ where  
\begin{align}
  x_i = i h && t_j = jk
\end{align}
Here $h = \Delta x$ is the grid cell spacing and $k = \Delta t$ is the time step, with $U^i_j \approx u(x_i, t_j)$ is numerical approximation at $(x_i, t_j)$.  

__Foward-Time Cenetered-Space (FTCS)__  

A first order approach, using centered differences in space 
\begin{align}  
  \frac{U_i^{j+1} - U_i^{j}}{\Delta t} = -\frac{a}{2 \Delta x} \left( U_{i+1}^{j} + U_{i-1}^{j}\right)
\end{align}  
and forward differences in time, can be written as   
\begin{align}  
  U_i^{j+1} = U_i^{j} -\frac{a \Delta t}{2 \Delta x} \left( U_{i+1}^{j} + U_{i-1}^{j}\right) . 
\end{align} 
This method is not useful because of stability limitations and is only first order accurate. We will not use this in practice, but usefull for comparison to more accurate and stable methods.  

__Lax-Wendroff__   
This method is based on the Taylor series expansion  
\begin{align}
  u(x,t_{j+1}) = u(x,t_{j}) + \Delta u_t(x,t_n) + \frac12 (\Delta t)^2 u_{tt}(x,t_n) + \ldots.
  \label{eqn:taylor}
\end{align}  
From the governing equation we have $u_t = a u_x$, differentiating this with respect to $t$ gives  
\begin{align}  
  u_{tt} = -au_{xt} = a^2 u_{xx}
\end{align}
where we have used $u_{xt} = u_{tx} = (a u_x)_x $ based on equality of mixed derivatives. Subing these expressions for $u_t$ and $u_tt$ into Eqn. \ref{eqn:taylor} gives  
\begin{align}
  u(x,t_{j+1}) = u(x,t_{j}) + \Delta t \; au_x(x,t_n) + \frac12 (\Delta t)^2 a^2 u_{xx}(x,t_n) + \ldots.
\end{align}
Using only these first three terms and expanding the spatial derivatives using centered differences gives
\begin{align}  
  U_i^{j+1} = U_i^{j} -\frac{ \Delta t}{2 \Delta x} a \left( U_{i+1}^{j} + U_{i-1}^{j}\right) + \frac{(\Delta t)^2}{2 (\Delta x)^2} a^2 \left(U_{i-1}^{j} -2U_{i}^{j} + U_{i+1}^{j}\right). 
\end{align} 
the _Lax-Wendroff_ method, which is second order accurate. 


__Upwind__  
The above methods are symetric in space, but the advection equation is aysymettric depending on the sign of $a$; if $a > 0$ the solution moves to the right and if $a<0$ the solution moves to the left. Therefore, under special circumstances when the solution is asymetteric we can take advantage of this  
\begin{align}  
  U_i^{j+1} = U_i^{j} -\frac{a \Delta t}{\Delta x} \left( U_{i}^{j} + U_{i-1}^{j}\right) 
\end{align} 
or   
\begin{align}  
  U_i^{j+1} = U_i^{j} -\frac{a \Delta t}{\Delta x} \left( U_{i+1}^{j} + U_{i}^{j}\right) 
\end{align} 

__Beam-Splitting__

## Python Implementation

In [10]:
import numpy as np 
import matplotlib.pyplot as plt

In [11]:
def gauss(x, σ = 0.05, μ = 0.5):
    '''1-D Gaussian Curve
    
    Keyword arguments:
    σ -- σ^2 is the variance
    μ -- Mean (i.e. center) 
    '''
    return 1./(σ*(2*np.pi)**2)*np.exp(-0.5*((x-μ)/σ)**2)

########################################################
#################   Init. Constant   ###################
########################################################
a  = 3e-6                  # wave speed 
L  = 1.                    # Domain Length 
nx = 100                   # Num. grid cells
dx = L/(nx-1)              # grid spacing

nt = 100                   # Num time steps
σ  = .75                   # courant number
dt = (σ*dx)**2/κ           # time step 

########################################################
##################   Init. Domain   ####################
########################################################
r  = (κ*dt)/(2*dx**2)      # matrix const.
x  = np.linspace(dx,L,nx)  # spatial grid
u  = np.zeros((5,nx,nt+1)) # (num methods) X (nx) X (nt)
u[:,:,0] = gauss(x)        # init. condition